In [1]:
import math
import random

### Thêm thư viện cần thiết để sử dụng tính toán

- `import math` được sử dụng trong tính toán liên quan đến toán.

- `import random` được sử dụng để tạo số ngẫu nhiên.

In [2]:
# Dữ liệu huấn luyện
X = [[1, 1], [2, 2], [2, 3], [3, 3]]  # Đầu vào
y_true = [2, 4, 5, 6]  # Đầu ra mục tiêu

### Thêm dữ liệu input và giá trị mong muốn mô hình dự đoán

In [3]:
# Hàm kích hoạt sigmoid
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(x):
    sig = sigmoid(x)
    return sig * (1 - sig)

### Xây dựng hàm kích hoạt Sigmoid và đạo hàm Sigmoid (dùng trong tính toán Gradient)

  - Công thức Sigmoid: $$y_{pred}(x) = \frac{1}{1+e^{-x}}$$

  - Đạo hàm Sigmoid: $$y_{pred}(1-y_{pred})$$

In [4]:
# Hàm chuẩn hóa và khôi phục giá trị
def normalize(data, min_val, max_val):
    return [(d - min_val) / (max_val - min_val) for d in data]

def denormalize(data, min_val, max_val):
    return [d * (max_val - min_val) + min_val for d in data]


# Chuẩn hóa dữ liệu
x_min = min(min(row) for row in X)
x_max = max(max(row) for row in X)
X_normalized = [[(xi - x_min) / (x_max - x_min) for xi in row] for row in X]

y_min, y_max = min(y_true), max(y_true)
y_true_normalized = normalize(y_true, y_min, y_max)

### Xây dựng các hàm và thao tác chuẩn hóa dữ liệu. 

  - Do giá trị output $y_{pred}$ của các nơ-ron có giá trị thuộc khoảng (0,1) nên nếu không chuẩn hóa dữ liệu huấn luyện thì sẽ đẫn đến sai sót làm mô hình không thể dự đoán chính xác.

  - Cơ chế hàm chuyển hóa là tùy người lập trình viên thiết lập.

  - Hàm **normalize** được dùng để chuẩn hóa dữ liệu thành giá trị thuộc khoảng (0,1).

  - Hàm **denormalize** được dùng để chuẩn hóa dữ liệu từ các giá trị thuộc khoảng (0,1) thành giá trị dữ liệu ban đầu.

In [5]:
# Khởi tạo trọng số và bias
random.seed(42)
weights_input_hidden = [[random.uniform(-0.5, 0.5) for _ in range(2)] for _ in range(4)]
bias_hidden = [random.uniform(-0.5, 0.5) for _ in range(4)]
weights_hidden_output = [random.uniform(-0.5, 0.5) for _ in range(4)]
bias_output = random.uniform(-0.5, 0.5)

### Khởi tạo các trọng số $w$ và hệ số bias ban đầu một cách ngẫu nhiên (sử dụng random)

- `random.seed(42)`

  - Đặt giá trị seed cho random, đảm bảo ở các lần khởi chạy lại chương trình thì kết quả là giống nhau.

- `weights_input_hidden = [[random.uniform(-0.5, 0.5) for _ in range(2)] for _ in range(4)]` và `bias_hidden = [random.uniform(-0.5, 0.5) for _ in range(4)]`

  - Khởi tạo các trọng số $w$ cho dữ liệu đầu vào input.

  - Như đã nói ở trên, do mỗi dữ liệu đầu vào $x$ có dạng là một mảng gồm 2 phần tử $x=[value_1,value_2]$ nên các giá trị trọng số cũng phải có dạng như vậy.

  - Mảng weights_input_hidden sẽ có dạng: $[ [w_{11},w_{12}] , [w_{21},w_{22}] , [w_{31},w_{32}] , [w_{41},w_{41}] ]$.

  - Mảng bias_hidden sẽ có dạng: $[b_1,b_2,b_3,b_4]$.

- `weights_hidden_output = [random.uniform(-0.5, 0.5) for _ in range(4)]` và `bias_output = random.uniform(-0.5, 0.5)`

  - Khởi tạo các giá trị trọng số $w$ và hệ số bias $b$ ngẫu nhiên cho lớp layer output.

  - Do giá trị đầu ra của nơ-ron thuộc lớp trước đó lần lượt là 4 giá trị đơn $y_{pred-1},y_{pred-2},y_{pred-3},y_{pred-4}$ (không còn là một mảng nữa) nên trọng số đầu ra sẽ có dạng $[w_1,w_2,w_3,w_4]$.

- **Lưu ý**: do chỉ thiết kế một lớp nơ-ron thuộc Hidden Layers nên bên trên chỉ khởi tạo một **weights_input_hidden** và **bias_hidden**.

In [6]:
# Tốc độ học
learning_rate = 0.02
epochs = 30000

### Khai báo tốc độ học `learning_rate` và số lần học của mô hình `epochs`.

In [2]:
for epoch in range(epochs):
    total_loss = 0

    for x, y in zip(X_normalized, y_true_normalized):
        # Lan truyền xuôi
        hidden_inputs = [sum(w * xi for w, xi in zip(weights, x)) + b for weights, b in zip(weights_input_hidden, bias_hidden)]
        
        hidden_outputs = [sigmoid(h) for h in hidden_inputs]
        
        output_input = sum(w * h for w, h in zip(weights_hidden_output, hidden_outputs)) + bias_output
        
        output = sigmoid(output_input)

        # Tính lỗi
        loss = (y - output) ** 2
        
        total_loss += loss

        # Lan truyền ngược
        output_error = 2 * (output - y) * sigmoid_derivative(output_input)
        
        grad_weights_hidden_output = [output_error * h for h in hidden_outputs]
        
        grad_bias_output = output_error

        
        hidden_errors = [output_error * w * sigmoid_derivative(h) for w, h in zip(weights_hidden_output, hidden_inputs)]
        
        grad_weights_input_hidden = [[he * xi for xi in x] for he in hidden_errors]
        
        grad_bias_hidden = hidden_errors

        # Cập nhật trọng số và bias
        weights_hidden_output = [w - learning_rate * gw for w, gw in zip(weights_hidden_output, grad_weights_hidden_output)]
        bias_output -= learning_rate * grad_bias_output
        
        for i in range(4):
            weights_input_hidden[i] = [w - learning_rate * gw for w, gw in zip(weights_input_hidden[i], grad_weights_input_hidden[i])]
            bias_hidden[i] -= learning_rate * grad_bias_hidden[i]

    # In lỗi sau mỗi 1000 epoch
    if (epoch + 1) % 1000 == 0:
        predictions = []
        for x in X_normalized:
            hidden_inputs = [sum(w * xi for w, xi in zip(weights, x)) + b for weights, b in zip(weights_input_hidden, bias_hidden)]
            hidden_outputs = [sigmoid(h) for h in hidden_inputs]
            output_input = sum(w * h for w, h in zip(weights_hidden_output, hidden_outputs)) + bias_output
            output = sigmoid(output_input)
            prediction = denormalize([output], y_min, y_max)[0]
            predictions.append(prediction)
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(X):.4f}, Predictions: {predictions}")


NameError: name 'epochs' is not defined

Giải thích từng phần trong vòng lặp huấn luyện mô hình:

In [ ]:
for epoch in range(epochs):
    total_loss = 0

- `for epoch in range(epochs):`

  - Vòng lặp huấn luyện mô hình (số lần mà mô hình thực hiện lan truyền xuôi và lan truyền ngược để cập nhật trọng số).

- `total_loss = 0`

  - Biến được dùng để tính tổng giá trị của hàm mất mát sau 1000 lần học.

In [ ]:
for epoch in range(epochs):
    total_loss = 0
    
    for x, y in zip(X_normalized, y_true_normalized):
           pass

- `for x, y in zip(X_normalized, y_true_normalized):`

  - Sử dụng zip để lần lượt gọi các phần tử bên trong mảng X_normalized - mảng chứa giá trị chuẩn hóa của input đầu vào; và y_true_normalized - mảng chứa các giá trị chuẩn hóa của output mong đợi.

  - 2 mảng đã được khởi tạo trước đó. [Xem](#khởi-tạo-các-trọng-số--và-hệ-số-bias-ban-đầu-một-cách-ngẫu-nhiên-sử-dụng-random)

- Bên dưới đây là các bước thực hiện trong **một lần học** của mô hình.

In [9]:
for x, y in zip(X_normalized, y_true_normalized):
        
       # Lan truyền xuôi
       hidden_inputs = [sum(w * xi for w, xi in zip(weights, x)) + b for weights, b in zip(weights_input_hidden, bias_hidden)]
        
       hidden_outputs = [sigmoid(h) for h in hidden_inputs]
        
       output_input = sum(w * h for w, h in zip(weights_hidden_output, hidden_outputs)) + bias_output
        
       output = sigmoid(output_input)
       
       print(hidden_inputs)

[1.327663009168956, 0.8718438627004594, -1.6726036719769615, -0.28216647745914164]
[-0.3114636004441813, -0.42918582090568536, -0.09331510060380377, 0.4616524996507629]
[-0.974953113224736, -0.8358772066799258, 0.2928007609893408, 0.4882930409359851]
[-1.9505902100573187, -1.7302155045118301, 1.485973470769354, 1.2054714767606673]


- **Xây dựng lan truyền xuôi**

  - `hidden_inputs`: Là mảng chứa các giá trị đầu ra $y$ của hàm tính toán giá trị đầu vào bên trong nơ-ron. $[y_1,y_2,y_3,y_4]$

    - Các phần tử trong mảng sẽ được tính lần lượt theo công thức của hàm tính toán đầu vào được nhắc bên trên.

    - Các phần tử trong mảng sẽ được tính như sau: 
    
      $y_1$ = weights_input_hidden[0][0] $\times$ X_normalized[0][0] + weights_input_hidden[0][1] $\times$ X_ normalized[0][1] + bias_hidden[0] 

      $y_2$ = weights_input_hidden[1][0] $\times$ X_normalized[1][0] + weights_input_hidden[1][1] $\times$ X_ normalized[1][1] + bias_hidden[1]

      $y_3$ = weights_input_hidden[2][0] $\times$ X_normalized[2][0] + weights_input_hidden[2][1] $\times$ X_ normalized[2][1] + bias_hidden[2]

      $y_4$ = weights_input_hidden[3][0] $\times$ X_normalized[3][0] + weights_input_hidden[3][1] $\times$ X_ normalized[3][1] + bias_hidden[3]

    - Như vậy là lớp nơ-ron này sẽ có 4 nơ-ron bên trong và mỗi nơ-ron nhận một giá trị input $x$.

  - `hidden_output`: Là mảng chứa các giá trị đầu ra $y_{pred}$ của các nơ-ron sau khi các giá trị $y$ bên trên được đi qua hàm kích hoạt Sigmoid. $[y_{pred-1},y_{pred-2},y_{pred-3},y_{pred-4}]$

    - Các phần tử trong mảng sẽ được tính như sau: 
      
      $y_{pred-1}$ = Sigmoid($y_1$)

      $y_{pred-2}$ = Sigmoid($y_2$)

      $y_{pred-3}$ = Sigmoid($y_3$)

      $y_{pred-4}$ = Sigmoid($y_4$)

  - `output_input`: Là biến chứa các giá trị đầu ra $o$ của hàm tính toán giá trị đầu vào bên trong nơ-ron. Đây là nơ-ron trong lớp Ouput Layer.

    - Giá trị $o$ sẽ được tính toán như sau: 

      $o$ = weights_hidden_output[0] $\times$ hidden_outputs[0] + weights_hidden_output[1] $\times$ hidden_outputs[1] + weights_hidden_output[2] $\times$ hidden_outputs[2] + weights_hidden_output[3] $\times$ hidden_outputs[3] + bias_output

    - Như vậy ta thấy lớp **Output Layer chỉ có một nơ-ron trong lớp này và nơ-ron này nhận 4 giá trị input đầu vào** $[y_{pred-1},y_{pred-2},y_{pred-3},y_{pred-4}]$.

  - `output`: Là biến lưu kết quả output đầu ra $o_{pred}$ của nơ-ron thuộc lớp Output Layers. Sau khi $o$ đi qua hàm kích hoạt Sigmoid.

    $o_{pred}$ = Sigmoid($o$)



In [ ]:
for x, y in zip(X_normalized, y_true_normalized):
       
       # Lan truyền xuôi
       hidden_inputs = [sum(w * xi for w, xi in zip(weights, x)) + b for weights, b in zip(weights_input_hidden, bias_hidden)]
        
       hidden_outputs = [sigmoid(h) for h in hidden_inputs]
        
       output_input = sum(w * h for w, h in zip(weights_hidden_output, hidden_outputs)) + bias_output
        
       output = sigmoid(output_input)

       # Tính lỗi
       loss = (y - output) ** 2
        
       total_loss += loss

- **Tính toán hàm lỗi**

  - `loss = (y - output) ** 2` 

    - Là biến lưu kết quả tính lỗi dựa trên kết quả đầu ra của Output Layer với giá trị mà ta mong đợi mô hình dự đoán. 
  
    - Sử dụng công thức hàm mất mát $$L (Loss) = \frac{1}{n} \sum_{i=1}^{n} (y_{true,i} - y_{pred,i})^2$$ 

    - Do ở đây ta chỉ tính tổng chứ không tính trung bình nên không chia cho $n$.

  - `total_loss += loss`

    - Là biến tính tổng lỗi sau một 1000 lần học.

In [ ]:
for x, y in zip(X_normalized, y_true_normalized):
        # Lan truyền xuôi
        hidden_inputs = [sum(w * xi for w, xi in zip(weights, x)) + b for weights, b in zip(weights_input_hidden, bias_hidden)]
        
        hidden_outputs = [sigmoid(h) for h in hidden_inputs]
        
        output_input = sum(w * h for w, h in zip(weights_hidden_output, hidden_outputs)) + bias_output
        
        output = sigmoid(output_input)

        # Tính lỗi
        loss = (y - output) ** 2
        
        total_loss += loss

        # Lan truyền ngược
        output_error = 2 * (output - y) * sigmoid_derivative(output_input)
        
        grad_weights_hidden_output = [output_error * h for h in hidden_outputs]
        
        grad_bias_output = output_error

        
        hidden_errors = [output_error * w * sigmoid_derivative(h) for w, h in zip(weights_hidden_output, hidden_inputs)]
        
        grad_weights_input_hidden = [[he * xi for xi in x] for he in hidden_errors]
        
        grad_bias_hidden = hidden_errors

- **Tính lan truyền ngược và thực hiện cập nhật trọng số**

  - Ta có các công thức liên quan

    - Gradient của hàm $L$ với trọng số $w_i$: $$\frac{\partial L}{\partial w_i} = \frac{2}{n} (y_{pred} - y_{true}) \times y_{pred}(1-y_{pred}) \times x_i$$

    - Gradient của hàm $L$ với hệ số bias: $$\frac{\partial L}{\partial w_0} = \frac{2}{n} (y_{pred} - y_{true}) \times y_{pred}(1-y_{pred})$$
  
  - `output_error`

    - Là biến được dùng để lưu giá trị của phép tính $\frac{2}{n} (y_{pred} - y_{true}) \times y_{pred}(1-y_{pred})$

    - Ở đây, ta không chia cho n do sử dụng thuật toán tối ưu SGD (chỉ cập nhật trên một mẫu riêng lẻ do đang trong vòng lặp for từng tập dữ liệu $x$ trong tập input đầu vào)

    - Nên `output_error` = ${2} \times (o_{pred} - y_{true}) \times o_{pred}(1-o_{pred})$

  - `grad_weights_hidden_output`

    - Là biến được dùng để lưu **giá trị tính Gradient của hàm mất mát so với các trọng số trong lớp nơ-ron của Output Layer** $\frac{\partial L}{\partial w_i}$

  - `grad_bias_output`

    - Là biến được dùng để lưu **giá trị tính Gradient của hàm mất mát so với hệ số bias trong lớp nơ-ron của Output Layer** $\frac{\partial L}{\partial w_0}$

  - `hidden_errors`

    - Công dụng tương tự với `output_error`.

  - `grad_weights_input_hidden`

    - Mảng dùng để lưu **giá trị tính Gradient của hàm mất mát so với các trọng số trong lớp nơ-ron của Hidden Layer** $\frac{\partial L}{\partial w_i}$

  - `grad_bias_hidden`

    - Là biến được dùng để lưu **giá trị tính Gradient của hàm mất mát so với hệ số bias trong lớp nơ-ron của Hidden Layer** $\frac{\partial L}{\partial w_0}$

In [ ]:
for x, y in zip(X_normalized, y_true_normalized):
        # Lan truyền xuôi
        hidden_inputs = [sum(w * xi for w, xi in zip(weights, x)) + b for weights, b in zip(weights_input_hidden, bias_hidden)]
        
        hidden_outputs = [sigmoid(h) for h in hidden_inputs]
        
        output_input = sum(w * h for w, h in zip(weights_hidden_output, hidden_outputs)) + bias_output
        
        output = sigmoid(output_input)

        # Tính lỗi
        loss = (y - output) ** 2
        
        total_loss += loss

        # Lan truyền ngược
        output_error = 2 * (output - y) * sigmoid_derivative(output_input)
        
        grad_weights_hidden_output = [output_error * h for h in hidden_outputs]
        
        grad_bias_output = output_error

        
        hidden_errors = [output_error * w * sigmoid_derivative(h) for w, h in zip(weights_hidden_output, hidden_inputs)]
        
        grad_weights_input_hidden = [[he * xi for xi in x] for he in hidden_errors]
        
        grad_bias_hidden = hidden_errors

        # Cập nhật trọng số và bias
        weights_hidden_output = [w - learning_rate * gw for w, gw in zip(weights_hidden_output, grad_weights_hidden_output)]
        bias_output -= learning_rate * grad_bias_output
        
        for i in range(4):
            weights_input_hidden[i] = [w - learning_rate * gw for w, gw in zip(weights_input_hidden[i], grad_weights_input_hidden[i])]
            bias_hidden[i] -= learning_rate * grad_bias_hidden[i]

- Cập nhật trọng số $w$ và hệ số bias

  - `weights_hidden_output` và `bias_output`

    - Cập nhật trọng số và hệ số bias cho lơp nơ-ron trong Output Layer.

  - `weights_input_hidden` và `bias_hidden`

    - Cập nhật các trọng số và hệ số bias cho lớp nơ-ron trong Hidden Layer, được xử lý trong vòng lặp for.

In [ ]:
for x, y in zip(X_normalized, y_true_normalized):
        # Lan truyền xuôi
        hidden_inputs = [sum(w * xi for w, xi in zip(weights, x)) + b for weights, b in zip(weights_input_hidden, bias_hidden)]
        
        hidden_outputs = [sigmoid(h) for h in hidden_inputs]
        
        output_input = sum(w * h for w, h in zip(weights_hidden_output, hidden_outputs)) + bias_output
        
        output = sigmoid(output_input)

        # Tính lỗi
        loss = (y - output) ** 2
        
        total_loss += loss

        # Lan truyền ngược
        output_error = 2 * (output - y) * sigmoid_derivative(output_input)
        
        grad_weights_hidden_output = [output_error * h for h in hidden_outputs]
        
        grad_bias_output = output_error

        
        hidden_errors = [output_error * w * sigmoid_derivative(h) for w, h in zip(weights_hidden_output, hidden_inputs)]
        
        grad_weights_input_hidden = [[he * xi for xi in x] for he in hidden_errors]
        
        grad_bias_hidden = hidden_errors

        # Cập nhật trọng số và bias
        weights_hidden_output = [w - learning_rate * gw for w, gw in zip(weights_hidden_output, grad_weights_hidden_output)]
        bias_output -= learning_rate * grad_bias_output
        
        for i in range(4):
            weights_input_hidden[i] = [w - learning_rate * gw for w, gw in zip(weights_input_hidden[i], grad_weights_input_hidden[i])]
            bias_hidden[i] -= learning_rate * grad_bias_hidden[i]

    # In lỗi sau mỗi 1000 epoch
if (epoch + 1) % 1000 == 0:
        predictions = []
        for x in X_normalized:
            hidden_inputs = [sum(w * xi for w, xi in zip(weights, x)) + b for weights, b in zip(weights_input_hidden, bias_hidden)]
            hidden_outputs = [sigmoid(h) for h in hidden_inputs]
            output_input = sum(w * h for w, h in zip(weights_hidden_output, hidden_outputs)) + bias_output
            output = sigmoid(output_input)
            prediction = denormalize([output], y_min, y_max)[0]
            predictions.append(prediction)
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(X):.4f}, Predictions: {predictions}")

- In các thông tin liên quan đến các giá trị output của mô hình và tổng lỗi sau mỗi 1000 lần học.

In [ ]:
# Kiểm tra kết quả
print("\nFinal Predictions:")
for x,x_input in zip(X_normalized,X):
    hidden_inputs = [sum(w * xi for w, xi in zip(weights, x)) + b for weights, b in zip(weights_input_hidden, bias_hidden)]
    hidden_outputs = [sigmoid(h) for h in hidden_inputs]
    output_input = sum(w * h for w, h in zip(weights_hidden_output, hidden_outputs)) + bias_output
    output = sigmoid(output_input)
    prediction = denormalize([output], y_min, y_max)[0]
    print(f"Input: {x_input}, Predicted Output: {prediction:.2f}")

- Sau khi đã kết thúc số lần học mà ta thiết lập, ta tiến hành kiểm tra dữ liệu output của mô hình xem có gần với giá trị mà ta mong đợi không